In [1]:
# ! pip install networkx
# ! pip install pybind11
# ! pip install graph-walker
import pandas as pd
import networkx as nx
# import walker

In [2]:
from model.post import Post, PostType

In [3]:
prep_df = pd.read_json("data/preprocessed_data.jsonl", orient="records")

posts = prep_df.to_dict("records")

assert len(posts) == len(prep_df), "Unexpectedly dropped records"

In [5]:
def unmarshal_post(record):
    return Post(
        post_id=record["index"],
        blog_url=record["blog_url"],
        post_type=PostType[record["type"].upper()],
        lang=record["lang"],
        is_reblog=(int(record["is_reblog"]) == 1),
        tags=record["tags"],
        root_tags=record["root_tags"]
    )

model_posts = list(map(unmarshal_post, posts))

In [142]:
from collections import defaultdict

co_occurring_tags = defaultdict(float)
for post in toy_posts:
    for i in range(len(post.tags)):
        for j in range(len(post.tags)):
            if i == j:
                continue
            else:
                key = tuple(sorted([post.tags[i], post.tags[j]]))
                co_occurring_tags[key] += (1 / len(post.tags))

In [217]:
post2post = nx.Graph()

for post in toy_posts:
    sorted_tags = sorted(post.tags)
    num_tags = len(sorted_tags)
    weight = 1 / num_tags
    for i in range(num_tags):
        for j in range(i+1, num_tags):
            
            a, b = sorted_tags[i], sorted_tags[j]
            if (a, b) in post2post.edges:
                new_weight = post2post.edges[a, b]['weight'] + weight
                post2post.add_edge(a, b, weight=new_weight)
            else:
                post2post.add_edge(a, b, weight=weight)

In [219]:
G = post2post

In [230]:
def prune(G, key, k = 25):
    return sorted(
        G[key].items(), 
        key=lambda x: x[1]["weight"], 
        reverse=True
    )[:k]

def random_sample(G, key, k = 25):
    pruned = prune(G, key, k)
    pop, weights = zip(*[
        (neighbor, features['weight'])
        for neighbor, features in pruned
    ])
    return random.choices(pop, weights, k=k)

random_sample(G, "star wars")

['din djarin',
 'din djarin',
 'sw',
 'the mandalorian',
 'q',
 'obi wan kenobi',
 'gfx',
 'art',
 'luke skywalker',
 'fanart',
 'rey',
 'fanart',
 'sequel trilogy',
 'rogue one',
 'daisy ridley',
 'art',
 'q',
 'din djarin',
 'finn',
 'luke skywalker',
 'luke skywalker',
 'the mandalorian',
 'the mandalorian',
 'reblog',
 'movies']

In [221]:
pruned = sorted(list(G["star wars"].items()), key=lambda x: x[1], reverse=True)

pop, weights = zip(*[
    (neighbor, features['weight'])
    for neighbor, features in pruned
])

TypeError: '<' not supported between instances of 'dict' and 'dict'

In [197]:
random.choices(pop, weights, k=10)

['din djarin',
 'darth maul',
 'q',
 'q',
 'rey',
 'movies',
 'sequel trilogy',
 'q',
 'art',
 'the mandalorian']

In [201]:
random.choice(posts)

{'index': 66702,
 'blog_url': 'tintuchotnhat27768.tumblr.com',
 'dt': '2021-06-15',
 'type': 'link',
 'lang': 'en_US',
 'is_reblog': 0,
 'tags': ['news'],
 'root_tags': []}

In [11]:
quota = 10
count = 0
for post in model_posts:
    if len(post.root_tags) == 0:
        if quota > 0:
            print(post)
            quota -= 1
        count += 1
count, count / len(model_posts)

Post(post_id=0, blog_url='fireland.tumblr.com', post_type=<PostType.PHOTO: 1>, lang='en_US', is_reblog=False, tags=['jfk', 'joe pesci', 'moviegifs', 'filmgifs'], root_tags=[])
Post(post_id=1, blog_url='erina.tumblr.com', post_type=<PostType.PHOTO: 1>, lang='en_US', is_reblog=False, tags=['she and Seven got along great!', 'seven doesn’t usually play well with other dogs'], root_tags=[])
Post(post_id=2, blog_url='goran.tumblr.com', post_type=<PostType.LINK: 5>, lang='en_US', is_reblog=False, tags=['classifieds’ saab of the day'], root_tags=[])
Post(post_id=3, blog_url='paintbynumbers.tumblr.com', post_type=<PostType.VIDEO: 3>, lang='en_US', is_reblog=False, tags=['Youtube'], root_tags=[])
Post(post_id=4, blog_url='alteralter.tumblr.com', post_type=<PostType.REGULAR: 2>, lang='en_US', is_reblog=False, tags=['diary'], root_tags=[])
Post(post_id=5, blog_url='shimuran.tumblr.com', post_type=<PostType.PHOTO: 1>, lang='en_US', is_reblog=False, tags=['火曜日', '塩大福', 'みずの'], root_tags=[])
Post(pos

(40906, 0.40906409064090643)